In [6]:
from pathlib import Path
import csv
from collections import defaultdict
import os.path
from statistics import mean 
import pandas as pd
import altair as alt
import fiona
import geopandas as gpd
import gpdvega
import json

from matplotlib import cm
from matplotlib.colors import to_hex

colormap = cm.spring

In [2]:
alt.renderers.enable('notebook')
alt.themes.enable('opaque')
alt.data_transformers.enable('gpd_to_values')

DataTransformerRegistry.enable('gpd_to_values')

In [3]:
def read_csv(filename):
    with open(filename, newline='') as File:  
        reader = csv.reader(File)
        header1 = next(reader)
        header2 = next(reader)
        results = []
        for row in reader:
            folder = Path(row[0].strip("'"))
            parts = folder.parts
            if len(parts) > 2 and parts[-1].lower().endswith('.tif'):
                results.append({"folder": str(folder.parent), "hits": int(row[1]), "bytes": int(row[2]), "cost": float(row[3])})
        return results

In [7]:
def read_json(filename):
    with open(filename, encoding='utf-8') as File:  
        reader = json.loads(File.read())
        reader_files = reader['Files']
        results = []
        for i,v in reader_files.items():
            if str(i).endswith('.TIF') or str(i).endswith('.tif') or str(i).endswith('.tiff'):
                results.append({"folder": str(i), "hits": int(v[0]), "bytes": int(v[1])})
        return results
    

In [9]:
all_data = read_json("week201925.json")

In [10]:
all_data

[{'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_green.tif',
  'hits': 1907,
  'bytes': 1343757048},
 {'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_red.tif',
  'hits': 1805,
  'bytes': 1300606007},
 {'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_blue.tif',
  'hits': 1805,
  'bytes': 1234688567},
 {'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07/NBART/NBART_B04.TIF',
  'hits': 1386,
  'bytes': 1134821244},
 {'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-13/S2B_OPER_MSI_ARD_TL_EPAE_20190613T023435_A011840_T53LPG_N02.07/NBART/NBART_B02.TIF',
  'hits': 2762,
  'bytes': 1105459735},
 {'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07/NBART/NBART_B03.TIF',
  'hits': 1

In [59]:
for i,v in all_data.items():
    if str(i).endswith('.TIF') or str(i).endswith('.tif') or str(i).endswith('.tiff'):
        print (i)
        print(v[0])

L2/sentinel-2-nrt/S2MSIARD/2019-06-17/S2A_OPER_MSI_ARD_TL_SGS__20190617T013457_A020805_T55JGG_N02.07/NBART/NBART_B04.TIF
1204
L2/sentinel-2-nrt/S2MSIARD/2019-06-17/S2A_OPER_MSI_ARD_TL_SGS__20190617T013457_A020805_T55JGG_N02.07/NBART/NBART_B03.TIF
1204
L2/sentinel-2-nrt/S2MSIARD/2019-06-17/S2A_OPER_MSI_ARD_TL_SGS__20190617T013457_A020805_T55JGG_N02.07/NBART/NBART_B02.TIF
1198
L2/sentinel-2-nrt/S2MSIARD/2019-06-17/S2A_OPER_MSI_ARD_TL_SGS__20190617T013457_A020805_T55HFE_N02.07/NBART/NBART_B03.TIF
1219
L2/sentinel-2-nrt/S2MSIARD/2019-05-17/S2B_OPER_MSI_ARD_TL_SGS__20190517T025326_A011454_T52KHG_N02.07/NBART/NBART_B12.TIF
1393
L2/sentinel-2-nrt/S2MSIARD/2019-06-06/S2B_OPER_MSI_ARD_TL_SGS__20190606T025353_A011740_T52KHG_N02.07/NBART/NBART_B12.TIF
1341
L2/sentinel-2-nrt/S2MSIARD/2019-05-27/S2B_OPER_MSI_ARD_TL_SGS__20190527T025321_A011597_T52KHG_N02.07/NBART/NBART_B12.TIF
1378
L2/sentinel-2-nrt/S2MSIARD/2019-05-17/S2B_OPER_MSI_ARD_TL_SGS__20190517T025326_A011454_T52KHG_N02.07/NBART/NBART_B11.T

In [ ]:
all_entries = read_csv("Files_20190417_20190423.csv")

In [ ]:
all_entries = read_csv("Files_20190407_20190506.csv")

In [5]:
all_entries = read_json("week201925.json")

NameError: name 'read_json' is not defined

In [9]:
all_entries = read_json("20190618.json")

In [6]:
len(all_entries)

99

In [7]:
def product_name(folder):
    parts = Path(folder).parts
    if parts[0] == 'mangrove_cover':
        return parts[0]
    return os.path.join(*parts[:2])


def spatial_id(folder):
    parts = Path(folder).parts
    print(parts)
    print(parts[-1])
    if parts[-2] in ['NBAR', 'NBART', 'QA', 'SUPPLEMENTARY', 'LAMBERTIAN']:
        print("OK")
    if len(parts) > 2 and parts[0] == 'L2' and parts[1] == 'sentinel-2-nrt' and parts[-2] in ['NBAR', 'NBART', 'QA', 'SUPPLEMENTARY', 'LAMBERTIAN']:
        try:
            print(parts[-3].split("_")[-2][1:])
            return parts[-3].split("_")[-2][1:]
        except IndexError:
            print(folder)
    else:
        return '<none>'

def merge_pre(folder, dicts):
    return {
        'hits': max(int(d['hits']) for d in dicts),
        'bytes': sum(int(d['bytes']) for d in dicts),
        #'cost': sum(float(d['cost']) for d in dicts),
        'spatial_id': spatial_id(folder),
        'product': product_name(folder),
        'folder': folder
    }

def group(entry_list, key):
    lookup = defaultdict(list)
    
    for d in entry_list:
        lookup[d[key]].append(d)
        
    return lookup
    

def merge(dicts):
    return {
        'hits': sum(int(d['hits']) for d in dicts),
        'bytes': sum(int(d['bytes']) for d in dicts),
        #'cost': sum(float(d['cost']) for d in dicts)
    }

In [8]:
stage1 = [merge_pre(key, value) for key, value in group(all_entries, 'folder').items()]

('hltc', 'v2.0.0', 'composite', 'low-tide', 'lon_117', 'lat_-20', 'COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_green.tif')
COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_green.tif
('hltc', 'v2.0.0', 'composite', 'low-tide', 'lon_117', 'lat_-20', 'COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_red.tif')
COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_red.tif
('hltc', 'v2.0.0', 'composite', 'low-tide', 'lon_117', 'lat_-20', 'COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_blue.tif')
COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_blue.tif
('L2', 'sentinel-2-nrt', 'S2MSIARD', '2019-06-19', 'S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07', 'NBART', 'NBART_B04.TIF')
NBART_B04.TIF
OK
55HFB
('L2', 'sentinel-2-nrt', 'S2MSIARD', '2019-06-13', 'S2B_OPER_MSI_ARD_TL_EPAE_20190613T023435_A011840_T53LPG_N02.07', 'NBART', 'NBART_B02.TIF')
NBART_B02.TIF
OK
53LPG
('L2', 'sentinel-2-nrt', 'S2MSIARD', '2019-06-19', 'S2B_OPER_MSI_ARD_TL

In [9]:
stage1

[{'hits': 1907,
  'bytes': 1343757048,
  'spatial_id': '<none>',
  'product': 'hltc/v2.0.0',
  'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_green.tif'},
 {'hits': 1805,
  'bytes': 1300606007,
  'spatial_id': '<none>',
  'product': 'hltc/v2.0.0',
  'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_red.tif'},
 {'hits': 1805,
  'bytes': 1234688567,
  'spatial_id': '<none>',
  'product': 'hltc/v2.0.0',
  'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_blue.tif'},
 {'hits': 1386,
  'bytes': 1134821244,
  'spatial_id': '55HFB',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07/NBART/NBART_B04.TIF'},
 {'hits': 2762,
  'bytes': 1105459735,
  'spatial_id': '53LPG',
  'product': 'L2/sentinel-2-nrt',
  'f

In [10]:
sentinel2 = [d for d in stage1 if d['product'] == 'L2/sentinel-2-nrt']

In [11]:
s2_summary = {key: merge(value)['hits'] for key, value in group(sentinel2, 'spatial_id').items()}

In [12]:
s2_summary

{'55HFB': 4159,
 '53LPG': 2762,
 '56JMQ': 1418,
 '55JGG': 3699,
 '55HFE': 2961,
 '52KHG': 81278,
 '52KGG': 21725}

In [47]:
s2_summary['56HLH']

KeyError: '56HLH'

In [84]:
s2_summary_list = list(s2_summary.items())

In [85]:
s2_summary_list

[('55HFB', 4159),
 ('53LPG', 2762),
 ('56JMQ', 1418),
 ('55JGG', 3699),
 ('55HFE', 2961),
 ('52KHG', 81278),
 ('52KGG', 21725)]

In [14]:
s2_mgrs = [entry[0] for entry in s2_summary_list]
s2_hits = [entry[1] for entry in s2_summary_list]

In [15]:
hit_values = pd.DataFrame({'MGRS': s2_mgrs, 'hits': s2_hits})

In [29]:
with open('australian-mgrs-tiles.geojson') as fl:
    input_gj = json.load(fl)

In [30]:
feats = input_gj['features']

In [56]:
for feat in feats:
    print(feat["geometry"]['coordinates'])

[[[107.4059053046, -41.4951070527, 0.0], [108.7199670857, -41.5286954845, 0.0], [108.6843689071, -42.5168269107, 0.0], [107.3498806889, -42.4820618042, 0.0], [107.4059053046, -41.4951070527, 0.0]]]
[[[107.4545362985, -40.5964014683, 0.0], [108.75086522, -40.6289489934, 0.0], [108.7168490517, -41.6172572933, 0.0], [107.4009979599, -41.5835648642, 0.0], [107.4545362985, -40.5964014683, 0.0]]]
[[[107.5010503813, -39.6969698611, 0.0], [108.7804165396, -39.7285030569, 0.0], [108.7479035043, -40.7169864681, 0.0], [107.4498746808, -40.6843383269, 0.0], [107.5010503813, -39.6969698611, 0.0]]]
[[[108.3730161916, -46.9230216432, 0.0], [109.8147223574, -46.94703384029999, 0.0], [109.7923235645, -47.93480605909999, 0.0], [108.323429721, -47.909954456, 0.0], [108.3730161916, -46.9230216432, 0.0]]]
[[[108.4159348796, -46.0243533958, 0.0], [109.8341080897, -46.04762762250001, 0.0], [109.812763259, -47.0355628073, 0.0], [108.3686790116, -47.01147664359999, 0.0], [108.4159348796, -46.0243533958, 0.0]]]

[[[131.5426073723, -45.1249853114, 0.0], [132.9360604945, -45.0854181283, 0.0], [133.0056683561, -46.07135312840001, 0.0], [131.5876572906, -46.1122976354, 0.0], [131.5426073723, -45.1249853114, 0.0]]]
[[[131.5035481794, -44.2254354549, 0.0], [132.875703577, -44.1870826283, 0.0], [132.9421153444, -45.1732466524, 0.0], [131.5465258707, -45.2129345939, 0.0], [131.5035481794, -44.2254354549, 0.0]]]
[[[131.4662891411, -43.3262569713, 0.0], [132.8181237037, -43.2890817984, 0.0], [132.8815173458, -44.2754702953, 0.0], [131.5073103421, -44.3139409574, 0.0], [131.4662891411, -43.3262569713, 0.0]]]
[[[131.4306895838, -42.4263721229, 0.0], [132.7631041779, -42.390341466, 0.0], [132.8236405568, -43.3769503249, 0.0], [131.469858884, -43.4142391925, 0.0], [131.4306895838, -42.4263721229, 0.0]]]
[[[131.3966836473, -41.52632232, 0.0], [132.7105438302, -41.4914043206, 0.0], [132.7683734147, -42.4782294354, 0.0], [131.4340988503, -42.5143705673, 0.0], [131.3966836473, -41.52632232, 0.0]]]
[[[131.364208

[[[135.93177461, -15.3746479288, 0.0], [136.9545477964, -15.3680206379, 0.0], [136.9641330541, -16.3600865007, 0.0], [135.9363468675, -16.3671641831, 0.0], [135.93177461, -15.3746479288, 0.0]]]
[[[135.927888065, -14.4704580539, 0.0], [136.9463999967, -14.4642374753, 0.0], [136.9553741558, -15.4563888544, 0.0], [135.9321687895, -15.4630560918, 0.0], [135.927888065, -14.4704580539, 0.0]]]
[[[135.9242637225, -13.5667403277, 0.0], [136.9388018264, -13.5609230676, 0.0], [136.9471772283, -14.5531552832, 0.0], [135.9282588075, -14.5594157307, 0.0], [135.9242637225, -13.5667403277, 0.0]]]
[[[135.9208924334, -12.6624139086, 0.0], [136.931734117, -12.6569972862, 0.0], [136.9395214778, -13.6493056879, 0.0], [135.9246069966, -13.6551622758, 0.0], [135.9208924334, -12.6624139086, 0.0]]]
[[[135.9177720283, -11.7580253441, 0.0], [136.9251923305, -11.753006658, 0.0], [136.9324018589, -12.7453864813, 0.0], [135.9212109446, -12.7508421555, 0.0], [135.9177720283, -11.7580253441, 0.0]]]
[[[135.9149000982,

[[[152.9997723645, -37.9475895716, 0.0], [154.2493542916, -37.9409562301, 0.0], [154.2665238389, -38.9302599114, 0.0], [152.9997692352, -38.9371311765, 0.0], [152.9997723645, -37.9475895716, 0.0]]]
[[[152.9997750848, -37.0460421885, 0.0], [154.2344292758, -37.0396201676, 0.0], [154.2508496818, -38.0290815842, 0.0], [152.999772092, -38.0357358602, 0.0], [152.9997750848, -37.0460421885, 0.0]]]
[[[152.9997776866, -36.1443574696, 0.0], [154.2201537941, -36.1381419059, 0.0], [154.235859512, -37.1277595041, 0.0], [152.9997748241, -37.1342019691, 0.0], [152.9997776866, -36.1443574696, 0.0]]]
[[[152.9997801738, -35.2430777118, 0.0], [154.2065067255, -35.2370638902, 0.0], [154.221530248, -36.2268358936, 0.0], [152.9997774357, -36.2330715605, 0.0], [152.9997801738, -35.2430777118, 0.0]]]
[[[152.9997825545, -34.3411223541, 0.0], [154.1934445404, -34.3353060526, 0.0], [154.2078153684, -35.3252307393, 0.0], [152.9997799353, -35.3312641037, 0.0], [152.9997825545, -34.3411223541, 0.0]]]
[[[152.999784

In [83]:
for feat in feats:
    polygon = Polygon(feat["geometry"]['coordinates'][0])
    print(polygon.centroid)
    print(polygon.centroid.y)
    
    print(feat["geometry"]['coordinates'])
    #a , b = (numpy.average(feat["geometry"]['coordinates'][0][0],feat["geometry"]['coordinates'][0][2]),numpy.average(feat["geometry"]['coordinates'][0][1],feat["geometry"]['coordinates'][0][3]) )

POINT (108.0400440194989 -42.00694592711531)
-42.00694592711531
[[[107.4059053046, -41.4951070527, 0.0], [108.7199670857, -41.5286954845, 0.0], [108.6843689071, -42.5168269107, 0.0], [107.3498806889, -42.4820618042, 0.0], [107.4059053046, -41.4951070527, 0.0]]]
POINT (108.0808290528198 -41.10777709177254)
-41.10777709177254
[[[107.4545362985, -40.5964014683, 0.0], [108.75086522, -40.6289489934, 0.0], [108.7168490517, -41.6172572933, 0.0], [107.4009979599, -41.5835648642, 0.0], [107.4545362985, -40.5964014683, 0.0]]]
POINT (108.1198313099915 -40.20789518986242)
-40.20789518986242
[[[107.5010503813, -39.6969698611, 0.0], [108.7804165396, -39.7285030569, 0.0], [108.7479035043, -40.7169864681, 0.0], [107.4498746808, -40.6843383269, 0.0], [107.5010503813, -39.6969698611, 0.0]]]
POINT (109.0758639070737 -47.43024270762326)
-47.43024270762326
[[[108.3730161916, -46.9230216432, 0.0], [109.8147223574, -46.94703384029999, 0.0], [109.7923235645, -47.93480605909999, 0.0], [108.323429721, -47.90995

[[[108.2775766286, -8.13290784, 0.0], [109.2735183181, -8.13836344, 0.0], [109.2689981242, -9.1310784816, 0.0], [108.2704553449, -9.1249470629, 0.0], [108.2775766286, -8.13290784, 0.0]]]
POINT (109.644839135021 -15.86847682260977)
-15.868476822609772
[[[109.1367159035, -15.3688028251, 0.0], [110.1595318708, -15.3750112207, 0.0], [110.1554075215, -16.3675521653, 0.0], [109.1275775588, -16.3609218439, 0.0], [109.1367159035, -15.3688028251, 0.0]]]
POINT (109.6506875748176 -14.96447268662353)
-14.964472686623534
[[[109.1444838043, -14.4649716614, 0.0], [110.1630376798, -14.4707990502, 0.0], [110.159176306, -15.4634215736, 0.0], [109.1359280735, -15.4571757562, 0.0], [109.1444838043, -14.4649716614, 0.0]]]
POINT (109.6561514646211 -14.06093723983586)
-14.06093723983586
[[[109.1517276966, -13.561609653, 0.0], [110.1663069712, -13.5670592145, 0.0], [110.1627032563, -14.5597589126, 0.0], [109.1437428122, -14.5538941748, 0.0], [109.1517276966, -13.561609653, 0.0]]]
POINT (109.6612446323655 -13.

POINT (116.4827530746645 -38.44065556321101)
-38.44065556321101
[[[115.8614709212, -37.9420809161, 0.0], [117.1110860058, -37.9475371324, 0.0], [117.1126131217, -38.9370768562, 0.0], [115.8458235572, -38.9314249343, 0.0], [115.8614709212, -37.9420809161, 0.0]]]
POINT (116.4890447143507 -37.53923829354558)
-37.53923829354558
[[[115.8750727166, -37.0407090266, 0.0], [117.109758542, -37.04599142, 0.0], [117.11121901, -38.0356832555, 0.0], [115.8601081077, -38.0302098193, 0.0], [115.8750727166, -37.0407090266, 0.0]]]
POINT (116.4950621399804 -36.63768179968881)
-36.637681799688814
[[[115.8880825401, -36.1391957622, 0.0], [117.1084888631, -36.1443083333, 0.0], [117.1098857498, -37.1341510389, 0.0], [115.8737692833, -37.1288518292, 0.0], [115.8880825401, -36.1391957622, 0.0]]]
POINT (116.5008144434246 -35.73652829852637)
-35.73652829852637
[[[115.9005196434, -35.238083543, 0.0], [117.1072750885, -35.2430301705, 0.0], [117.108611286, -36.2330222654, 0.0], [115.8868281231, -36.2278931582, 0.0]

POINT (115.6703324213897 -11.34884571104218)
-11.348845711042179
[[[115.1704427673, -10.8500436098, 0.0], [116.1747533017, -10.8543748365, 0.0], [116.1718957354, -11.8472792732, 0.0], [115.1641110414, -11.8425412436, 0.0], [115.1704427673, -10.8500436098, 0.0]]]
POINT (115.6743415956397 -10.44451492110213)
-10.444514921102133
[[[115.1757006196, -9.9458901596, 0.0], [116.1771262066, -9.9498530593, 0.0], [116.1745095596, -10.9428165691, 0.0], [115.1699026881, -10.938449216, 0.0], [115.1757006196, -9.9458901596, 0.0]]]
POINT (115.6780003505477 -9.540131565514532)
-9.540131565514532
[[[115.1804793074, -9.0416854472, 0.0], [116.1792828518, -9.0452819573, 0.0], [116.1769037689, -10.0382998453, 0.0], [115.1752077439, -10.0343010191, 0.0], [115.1804793074, -9.0416854472, 0.0]]]
POINT (115.6813114752912 -8.636242593105806)
-8.636242593105806
[[[115.1847824674, -8.1379764451, 0.0], [116.1812248811, -8.1412085325, 0.0], [116.1790801922, -9.1342760216, 0.0], [115.180030254, -9.1306435466, 0.0], [1

[[[119.8964632496, -29.7940071465, 0.0], [121.0315617773, -29.8158011617, 0.0], [121.0116635677, -30.80606353210001, 0.0], [119.8651279858, -30.7833923229, 0.0], [119.8964632496, -29.7940071465, 0.0]]]
POINT (120.4740645251273 -29.39888691591342)
-29.398886915913423
[[[119.9236797941, -28.8925935995, 0.0], [121.0488439861, -28.9136031144, 0.0], [121.0298292898, -29.904011405, 0.0], [119.893734922, -29.8821399496, 0.0], [119.9236797941, -28.8925935995, 0.0]]]
POINT (120.4958666001963 -28.49699062108997)
-28.496990621089974
[[[119.9496781546, -27.9910369633, 0.0], [121.0653521689, -28.0112750946, 0.0], [121.0471887242, -29.0018262192, 0.0], [119.9210730016, -28.9807405857, 0.0], [119.9496781546, -27.9910369633, 0.0]]]
POINT (120.5166761296062 -27.59550089040452)
-27.59550089040452
[[[119.9744868751, -27.0898818031, 0.0], [121.081104521, -27.1093614166, 0.0], [121.0637616322, -28.1000521008, 0.0], [119.9471732219, -28.0797386422, 0.0], [119.9744868751, -27.0898818031, 0.0]]]
POINT (120.53

...

POINT (147.5657493314135 -29.42163328798437)
-29.421633287984367
[[[146.9997948036, -28.9277411693, 0.0], [148.1262332689, -28.9230308491, 0.0], [148.137214549, -29.9138260419, 0.0], [146.9997928023, -29.918729695, 0.0], [146.9997948036, -28.9277411693, 0.0]]]
POINT (147.5608602478577 -28.51891077062626)
-28.518910770626263
[[[146.999796541, -28.0248938952, 0.0], [148.1166996733, -28.0203565874, 0.0], [148.1271892016, -29.0112881209, 0.0], [146.9997946294, -29.0160155141, 0.0], [146.999796541, -28.0248938952, 0.0]]]
POINT (147.5561938854607 -27.61660890606125)
-27.61660890606125
[[[146.9997981989, -27.1224696414, 0.0], [148.1076026908, -27.1181024529, 0.0], [148.1176182144, -28.1091674044, 0.0], [146.9997963736, -28.1137216068, 0.0], [146.9997981989, -27.1224696414, 0.0]]]
POINT (147.551736941692 -26.71364704030515)
-26.713647040305155
[[[146.9997997819, -26.2193876212, 0.0], [148.0989165525, -26.215188013, 0.0], [148.1084741432, -27.2063834976, 0.0], [146.9997980401, -27.2107672116, 0

-44.72964638186386
[[[154.252469411, -44.2460108766, 0.0], [155.6266219638, -44.2226701191, 0.0], [155.6717064934, -45.2100729321, 0.0], [154.2739912592, -45.23422702440001, 0.0], [154.252469411, -44.2460108766, 0.0]]]
POINT (154.9265640687943 -43.83023905677614)
-43.830239056776136
[[[154.2338121378, -43.3462000094, 0.0], [155.5875361841, -43.3235765959, 0.0], [155.630568566, -44.3111671202, 0.0], [154.2543533388, -44.3345796583, 0.0], [154.2338121378, -43.3462000094, 0.0]]]
POINT (154.8982584728721 -42.93013371077745)
-42.930133710777454
[[[154.2159865731, -42.4457006262, 0.0], [155.5501910406, -42.4237743128, 0.0], [155.5912809544, -43.4115506149, 0.0], [154.2355996342, -43.4342432806, 0.0], [154.2159865731, -42.4457006262, 0.0]]]
POINT (154.8712261154571 -42.02987131778056)
-42.02987131778056
[[[154.1989596317, -41.5450534441, 0.0], [155.5145174743, -41.5238047729, 0.0], [155.5537674855, -42.5117647877, 0.0], [154.2176936452, -42.5337583865, 0.0], [154.1989596317, -41.5450534441, 0

POINT (153.5103828927203 -14.96789109902292)
-14.967891099022916
[[[152.9998144137, -14.4722876312, 0.0], [154.0186257582, -14.4700827372, 0.0], [154.0233249228, -15.4626538253, 0.0], [152.9998135574, -15.465017044, 0.0], [152.9998144137, -14.4722876312, 0.0]]]
POINT (153.5083155032474 -14.0641407705433)
-14.0641407705433
[[[152.9998151387, -13.5684512784, 0.0], [154.014647135, -13.5663893456, 0.0], [154.0190327408, -14.5590380084, 0.0], [152.9998143396, -14.5612570345, 0.0], [152.9998151387, -13.5684512784, 0.0]]]
POINT (153.5063883922032 -13.15978004283123)
-13.159780042831233
[[[152.9998158132, -12.664007022, 0.0], [154.0109462998, -12.6620870993, 0.0], [154.0150239643, -13.6548089209, 0.0], [152.9998150701, -13.6568847938, 0.0], [152.9998158132, -12.664007022, 0.0]]]
POINT (153.504600302207 -12.25535540690051)
-12.255355406900506
[[[152.9998164374, -11.7595014154, 0.0], [154.0075208713, -11.7577225445, 0.0], [154.0112959457, -12.75051299, 0.0], [152.9998157494, -12.752446755, 0.0],

POINT (161.7858972391797 -34.80379370827934)
-34.803793708279336
[[[161.1733316344, -34.3218325277, 0.0], [162.3652377054, -34.2948655567, 0.0], [162.4056772124, -35.2832818792, 0.0], [161.1994838956, -35.3112545059, 0.0], [161.1733316344, -34.3218325277, 0.0]]]
POINT (161.7562229135401 -33.90284807622272)
-33.90284807622272
[[[161.150593851, -33.4203854124, 0.0], [162.3300764495, -33.3943145887, 0.0], [162.368760331, -34.3829129924, 0.0], [161.1756096687, -34.40996859, 0.0], [161.150593851, -33.4203854124, 0.0]]]
POINT (161.7278552698485 -33.00228880243096)
-33.00228880243096
[[[161.128858847, -32.5193348717, 0.0], [162.2964645039, -32.4941417267, 0.0], [162.3334667456, -33.482918337, 0.0], [161.1527862176, -33.509076512, 0.0], [161.128858847, -32.5193348717, 0.0]]]
POINT (161.7007072422647 -32.1010376969024)
-32.1010376969024
[[[161.1080603774, -31.6176017111, 0.0], [162.2642996159, -31.5932697937, 0.0], [162.2996874049, -32.5822209175, 0.0], [161.1309428833, -32.6074991701, 0.0], [1

-15.847372169463503
[[[161.7940776214, -15.3590716628, 0.0], [162.8153693683, -15.3439214673, 0.0], [162.8340391612, -16.3343498049, 0.0], [161.8077688246, -16.3505294006, 0.0], [161.7940776214, -15.3590716628, 0.0]]]
POINT (162.2984679083074 -14.94462629592588)
-14.944626295925884
[[[161.782439498, -14.4558376565, 0.0], [162.7994989295, -14.4416170627, 0.0], [162.8169789426, -15.4321444451, 0.0], [161.7952579669, -15.4473859426, 0.0], [161.782439498, -14.4558376565, 0.0]]]
POINT (162.2851355474161 -14.0423382759175)
-14.042338275917503
[[[161.7715863465, -13.5530678297, 0.0], [162.7846986813, -13.5397691098, 0.0], [162.8010128546, -14.5303899104, 0.0], [161.7835496823, -14.5447016318, 0.0], [161.7715863465, -13.5530678297, 0.0]]]
POINT (162.2727075129235 -13.13942974399865)
-13.139429743998647
[[[161.761490809, -12.6496830157, 0.0], [162.7709313673, -12.6373000692, 0.0], [162.7861004847, -13.6280087338, 0.0], [161.7726142939, -13.6413973473, 0.0], [161.761490809, -12.6496830157, 0.0]]

In [66]:
import numpy 
import shapely 

In [4]:
from shapely.geometry import Polygon
polygon = Polygon([[107.4059053046, -41.4951070527, 0.0],[108.7199670857, -41.5286954845, 0.0] ,[108.6843689071, -42.5168269107, 0.0],[107.3498806889, -42.4820618042, 0.0] ])
print(polygon.centroid)
print(polygon.centroid.coords)
dir(polygon.centroid)

POINT (108.0400440194989 -42.00694592711531)


['__and__',
 '__array_interface__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__geo_interface__',
 '__geom__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__p__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '__xor__',
 '_crs',
 '_ctypes_data',
 '_geom',
 '_get_coords',
 '_is_empty',
 '_lgeos',
 '_ndim',
 '_other_owned',
 '_repr_svg_',
 '_set_coords',
 'almost_equals',
 'area',
 'array_interface',
 'array_interface_base',
 'boundary',
 'bounds',
 'buffer',
 'centroid',
 'contains',
 'convex_hull',
 'coords',
 'covers',
 'crosses',
 'ctypes',
 'difference',
 'disjoint',
 'distance',
 'empty',
 'envelope',
 'equals',
 'equals_exact',
 'geom_type',
 'geometryType',
 'has_z',
 'hausdorff_distance',


In [31]:
for feat in feats:
    print(feat['properties'])
    if 'MGRS' in feat['properties']:
        mgrs = feat['properties']['MGRS']
        if mgrs in s2_summary:
            feat['properties']['hits'] = s2_summary[mgrs]
            feat['properties']['stroke'] = "#ff00ff"
            feat['properties']['fill-opacity'] = 0.5
            feat['properties']['fill'] = "#ff0000"
            if feat['properties']['hits'] > 100:
                feat['properties']['fill'] = "#0000ff"
                

{'MGRS': '49GBP'}
{'MGRS': '49GBQ'}
{'MGRS': '49GBR'}
{'MGRS': '49GCH'}
{'MGRS': '49GCJ'}
{'MGRS': '49GCK'}
{'MGRS': '49GCL'}
{'MGRS': '49GCM'}
{'MGRS': '49GCN'}
{'MGRS': '49GCP'}
{'MGRS': '49GCQ'}
{'MGRS': '49GCR'}
{'MGRS': '49GDH'}
{'MGRS': '49GDJ'}
{'MGRS': '49GDK'}
{'MGRS': '49GDL'}
{'MGRS': '49GDM'}
{'MGRS': '49GDN'}
{'MGRS': '49GDP'}
{'MGRS': '49GDQ'}
{'MGRS': '49GDR'}
{'MGRS': '49GEH'}
{'MGRS': '49GEJ'}
{'MGRS': '49GEK'}
{'MGRS': '49GEL'}
{'MGRS': '49GEM'}
{'MGRS': '49GEN'}
{'MGRS': '49GEP'}
{'MGRS': '49GEQ'}
{'MGRS': '49GER'}
{'MGRS': '49GFH'}
{'MGRS': '49GFJ'}
{'MGRS': '49GFK'}
{'MGRS': '49GFL'}
{'MGRS': '49GFM'}
{'MGRS': '49GFN'}
{'MGRS': '49GFP'}
{'MGRS': '49GFQ'}
{'MGRS': '49GFR'}
{'MGRS': '49GGH'}
{'MGRS': '49GGJ'}
{'MGRS': '49GGK'}
{'MGRS': '49GGL'}
{'MGRS': '49GGM'}
{'MGRS': '49GGN'}
{'MGRS': '49GGP'}
{'MGRS': '49GGQ'}
{'MGRS': '49GGR'}
{'MGRS': '49HBA'}
{'MGRS': '49HBB'}
{'MGRS': '49HBC'}
{'MGRS': '49HBD'}
{'MGRS': '49HBE'}
{'MGRS': '49HBS'}
{'MGRS': '49HBT'}
{'MGRS': '

{'MGRS': '53GQR'}
{'MGRS': '53HKA'}
{'MGRS': '53HKB'}
{'MGRS': '53HKC'}
{'MGRS': '53HKD'}
{'MGRS': '53HKE'}
{'MGRS': '53HKS'}
{'MGRS': '53HKT'}
{'MGRS': '53HKU'}
{'MGRS': '53HKV'}
{'MGRS': '53HLA'}
{'MGRS': '53HLB'}
{'MGRS': '53HLC'}
{'MGRS': '53HLD'}
{'MGRS': '53HLE'}
{'MGRS': '53HLS'}
{'MGRS': '53HLT'}
{'MGRS': '53HLU'}
{'MGRS': '53HLV'}
{'MGRS': '53HMA'}
{'MGRS': '53HMB'}
{'MGRS': '53HMC'}
{'MGRS': '53HMD'}
{'MGRS': '53HME'}
{'MGRS': '53HMS'}
{'MGRS': '53HMT'}
{'MGRS': '53HMU'}
{'MGRS': '53HMV'}
{'MGRS': '53HNA'}
{'MGRS': '53HNB'}
{'MGRS': '53HNC'}
{'MGRS': '53HND'}
{'MGRS': '53HNE'}
{'MGRS': '53HNS'}
{'MGRS': '53HNT'}
{'MGRS': '53HNU'}
{'MGRS': '53HNV'}
{'MGRS': '53HPA'}
{'MGRS': '53HPB'}
{'MGRS': '53HPC'}
{'MGRS': '53HPD'}
{'MGRS': '53HPE'}
{'MGRS': '53HPS'}
{'MGRS': '53HPT'}
{'MGRS': '53HPU'}
{'MGRS': '53HPV'}
{'MGRS': '53HQA'}
{'MGRS': '53HQB'}
{'MGRS': '53HQC'}
{'MGRS': '53HQD'}
{'MGRS': '53HQE'}
{'MGRS': '53HQS'}
{'MGRS': '53HQT'}
{'MGRS': '53HQU'}
{'MGRS': '53HQV'}
{'MGRS': '

{'MGRS': '54LUN'}
{'MGRS': '54LUP'}
{'MGRS': '54LUQ'}
{'MGRS': '54LUR'}
{'MGRS': '54LVH'}
{'MGRS': '54LVJ'}
{'MGRS': '54LVK'}
{'MGRS': '54LVL'}
{'MGRS': '54LVM'}
{'MGRS': '54LVN'}
{'MGRS': '54LVP'}
{'MGRS': '54LVQ'}
{'MGRS': '54LVR'}
{'MGRS': '54LWH'}
{'MGRS': '54LWJ'}
{'MGRS': '54LWK'}
{'MGRS': '54LWL'}
{'MGRS': '54LWM'}
{'MGRS': '54LWN'}
{'MGRS': '54LWP'}
{'MGRS': '54LWQ'}
{'MGRS': '54LWR'}
{'MGRS': '54LXH'}
{'MGRS': '54LXJ'}
{'MGRS': '54LXK'}
{'MGRS': '54LXL'}
{'MGRS': '54LXM'}
{'MGRS': '54LXN'}
{'MGRS': '54LXP'}
{'MGRS': '54LXQ'}
{'MGRS': '54LXR'}
{'MGRS': '54LYH'}
{'MGRS': '54LYJ'}
{'MGRS': '54LYK'}
{'MGRS': '54LYL'}
{'MGRS': '54LYM'}
{'MGRS': '54LYN'}
{'MGRS': '54LYP'}
{'MGRS': '54LYQ'}
{'MGRS': '54LYR'}
{'MGRS': '54LZH'}
{'MGRS': '54LZJ'}
{'MGRS': '54LZK'}
{'MGRS': '54LZL'}
{'MGRS': '54LZM'}
{'MGRS': '54LZN'}
{'MGRS': '54LZP'}
{'MGRS': '54LZQ'}
{'MGRS': '54LZR'}
{'MGRS': '54MTA'}
{'MGRS': '54MTB'}
{'MGRS': '54MTC'}
{'MGRS': '54MTD'}
{'MGRS': '54MTE'}
{'MGRS': '54MTS'}
{'MGRS': '

{'MGRS': '57KZA'}
{'MGRS': '57KZB'}
{'MGRS': '57KZP'}
{'MGRS': '57KZQ'}
{'MGRS': '57KZR'}
{'MGRS': '57KZS'}
{'MGRS': '57KZT'}
{'MGRS': '57KZU'}
{'MGRS': '57KZV'}
{'MGRS': '57LTC'}
{'MGRS': '57LTD'}
{'MGRS': '57LTE'}
{'MGRS': '57LTF'}
{'MGRS': '57LTG'}
{'MGRS': '57LTH'}
{'MGRS': '57LTJ'}
{'MGRS': '57LTK'}
{'MGRS': '57LTL'}
{'MGRS': '57LUC'}
{'MGRS': '57LUD'}
{'MGRS': '57LUE'}
{'MGRS': '57LUF'}
{'MGRS': '57LUG'}
{'MGRS': '57LUH'}
{'MGRS': '57LUJ'}
{'MGRS': '57LUK'}
{'MGRS': '57LUL'}
{'MGRS': '57LVC'}
{'MGRS': '57LVD'}
{'MGRS': '57LVE'}
{'MGRS': '57LVF'}
{'MGRS': '57LVG'}
{'MGRS': '57LVH'}
{'MGRS': '57LVJ'}
{'MGRS': '57LVK'}
{'MGRS': '57LVL'}
{'MGRS': '57LWC'}
{'MGRS': '57LWD'}
{'MGRS': '57LWE'}
{'MGRS': '57LWF'}
{'MGRS': '57LWG'}
{'MGRS': '57LWH'}
{'MGRS': '57LWJ'}
{'MGRS': '57LWK'}
{'MGRS': '57LWL'}
{'MGRS': '57LXC'}
{'MGRS': '57LXD'}
{'MGRS': '57LXE'}
{'MGRS': '57LXF'}
{'MGRS': '57LXG'}
{'MGRS': '57LXH'}
{'MGRS': '57LXJ'}
{'MGRS': '57LXK'}
{'MGRS': '57LXL'}
{'MGRS': '57LYC'}
{'MGRS': '

In [32]:
with open('test.geojson', 'w') as fl:
    json.dump(input_gj, fl)

In [86]:
input_gj

{'features': [{'geometry': {'coordinates': [[[107.4059053046,
       -41.4951070527,
       0.0],
      [108.7199670857, -41.5286954845, 0.0],
      [108.6843689071, -42.5168269107, 0.0],
      [107.3498806889, -42.4820618042, 0.0],
      [107.4059053046, -41.4951070527, 0.0]]],
    'type': 'Polygon'},
   'properties': {'MGRS': '49GBP'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[107.4545362985, -40.5964014683, 0.0],
      [108.75086522, -40.6289489934, 0.0],
      [108.7168490517, -41.6172572933, 0.0],
      [107.4009979599, -41.5835648642, 0.0],
      [107.4545362985, -40.5964014683, 0.0]]],
    'type': 'Polygon'},
   'properties': {'MGRS': '49GBQ'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[107.5010503813, -39.6969698611, 0.0],
      [108.7804165396, -39.7285030569, 0.0],
      [108.7479035043, -40.7169864681, 0.0],
      [107.4498746808, -40.6843383269, 0.0],
      [107.5010503813, -39.6969698611, 0.0]]],
    'type': 'Polygon'},
   'properties': {'MGRS'

In [88]:
type(feats)

list

In [ ]:
ex_feat = feats[0]

In [ ]:
list(ex_feat['properties'])

In [ ]:
with open('low_composite_20.geojson') as fl:
    low_comp = json.load(fl)

In [ ]:
list(low_comp['features'][0]['properties'])

In [ ]:
mgrs_poly = {}

In [ ]:
for entry in grid:
    mgrs = entry['properties']['MGRS']
    mgrs_poly[mgrs] = entry

In [ ]:
c234 = set(mgrs_poly)

In [ ]:
geojson2 = [mgrs_poly[x] for x in c234]

In [ ]:
common = set(s2_summary) & set(mgrs_poly)

In [ ]:
geojson = [mgrs_poly[x] for x in common]

In [ ]:
alt.Chart(alt.Data(values=geojson)).mark_geoshape(stroke='black', strokeWidth=1).encode().properties(width=400, height=400)

In [ ]:
polygons = gpd.GeoDataFrame.from_features((geojson))

In [ ]:
polygons.head()

In [ ]:
alt.Chart(merged).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )

In [ ]:
merged = polygons.merge(hit_values, on= 'MGRS', how = 'inner')[['geometry', 'hits', 'MGRS']]

In [ ]:
merged.head()

In [ ]:
def gen_map(geodata, color_column, title):
    '''Generates DC ANC map with population choropleth and ANC labels'''
    # Add Base Layer
    base = alt.Chart(geodata, title = title).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(
        fill='lightgray',
        stroke='black'
    ).encode(
        alt.Color(color_column, 
                  type='quantitative', 
                 scale=alt.Scale(scheme='bluegreen'),
                 title = "hits")
    )

    return base + choro


In [ ]:
gen_map(merged, 'hits', 'sentinel-2')

In [ ]:
gen_map(polygons, 'hits', 'sentinel-2')

In [ ]:

  
def gen_base(geojson):
    '''Generates baselayer of DC ANC map'''
    base = alt.Chart(alt.Data(values=geojson)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    return base



In [ ]:
import requests

In [ ]:
geos = "australia-mgrs-grid.geojson"

In [ ]:
gen_base(grid)

In [ ]:
polygons.plot()

In [ ]:
geojson

In [ ]:
gdf_js = gpd.read_file("australia-mgrs-grid.geojson")

In [ ]:
gdf_js

In [ ]:
gdf_js.shape

In [ ]:
gdf_js.plot()

In [ ]:
%matplotlib

In [ ]:
gdf_js.GZD.unique()

In [ ]:
from matplotlib import cm
from matplotlib.colors import to_hex

In [ ]:
to_hex(colormap(1))

In [ ]:
to_hex(colormap(0))

In [33]:
import datetime

In [34]:
today=datetime.date.today()
yesterday =(today - datetime.timedelta(days =2)).strftime("%Y%m%d")

In [35]:
today

datetime.date(2019, 6, 27)

In [36]:
yesterday

'20190625'

In [42]:
week = today.strftime("%Y%V")

In [43]:
week

'201926'

In [46]:
month= today.strftime("%Y%m")

In [47]:
month

'201906'